In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

In [14]:
#输入数据
f=open('E:\论文实现\数据\phoneme.data')
df=pd.read_csv(f)
df=df.drop('row.names',1)
df_train=df.head(1000)
df_test=df[1000:4509]
data_train=np.array(df_train)
data_test=np.array(df_test)
#将数据按类标签分成五类
df1=df_train.query('g=="sh"')
df2=df_train.query('g=="dcl"')
df3=df_train.query('g=="iy"')
df4=df_train.query('g=="aa"')
df5=df_train.query('g=="ao"')
X_train=np.array(df_train.drop(df_train.columns[[256,257]],1))
X_test=np.array(df_test.drop(df_test.columns[[256,257]],1))
Y_train=data_train[:,256]
Y_test=data_test[:,256]
X_sh=np.array(df1.drop(df1.columns[[256,257]],1))
X_dcl=np.array(df2.drop(df2.columns[[256,257]],1))
X_iy=np.array(df3.drop(df3.columns[[256,257]],1))
X_aa=np.array(df4.drop(df4.columns[[256,257]],1))
X_ao=np.array(df5.drop(df5.columns[[256,257]],1))

In [15]:
#计算F统计量，选出m个h-separared markers
X_mean=np.average(X_train,0)
X_sh_mean=np.average(X_sh,0)
X_dcl_mean=np.average(X_dcl,0)
X_iy_mean=np.average(X_iy,0)
X_aa_mean=np.average(X_aa,0)
X_ao_mean=np.average(X_ao,0)
# n1=len(X_sh)
# n2=len(X_dcl)
# n3=len(X_iy)
# n4=len(X_aa)
# n5=len(X_ao)
Sb=(len(X_sh)*np.square(X_sh_mean-X_mean)+len(X_dcl)*np.square(X_dcl_mean-X_mean)
    +len(X_iy)*np.square(X_iy_mean-X_mean)+len(X_aa)*np.square(X_aa_mean-X_mean)
    +len(X_ao)*np.square(X_ao_mean-X_mean))/4
Sw=(np.var(X_sh,axis=0)*len(X_sh)+np.var(X_dcl,axis=0)*len(X_dcl)
    +np.var(X_iy,axis=0)*len(X_iy)+len(X_aa)*np.var(X_aa,axis=0)
    +len(X_ao)*np.var(X_ao,axis=0))/(len(X_train)-5)
F=Sb/Sw
print(F)

[   32.10001301   106.79397131   312.20008159   691.86048197   286.0402012
   143.12096574   229.04272375   356.41780874   347.92652372   393.20276007
   453.31201517   489.87306869   414.78572186   461.25726472   598.68778372
   670.21439469   801.87736086   930.75678686  1051.73924113   971.02200498
  1020.75471759  1061.51952923  1022.49278194   971.12129221   978.37267356
   889.47328852   923.35355446  1138.32023034  1087.47922216  1167.38835256
   996.78989528  1085.86047165  1132.18816616  1009.23295103   973.14807829
   970.00679919   946.48663374   830.83414282   763.04618023   846.89476706
   716.62382735   641.35126693   598.24176813   588.34439835   508.50325132
   481.4749254    490.90867709   417.70642219   354.64883228   325.37584589
   326.73167078   335.19534365   266.08090116   271.5696453    281.72039785
   284.67515971   314.74356251   364.08604949   340.31934999   368.34319211
   403.47278098   453.59837058   484.66757098   430.21619569   480.48306432
   475.958172

In [16]:
#根据m,h选择m个markers
def findmarkers(m,h,F):
    markers=np.zeros(m)
    for i in range(m):
        markers[i]=np.argmax(F)
        F[max(np.argmax(F)-h,0):min(np.argmax(F)+h,len(F)-1)]=0
    return markers
m=7
h=6
markers=findmarkers(m,h,F)
print(markers)

[ 29.  21.  35.   3.  41.  14.  69.]


In [17]:
#LDA降维，得到3m个线性判别变量
lda = LinearDiscriminantAnalysis(n_components=3)
lda.fit(X_train[:,int(max(markers[0]-h,0)):int(min(markers[0]+h,X_train.shape[1]-1))],Y_train)
X_train_ldv = lda.transform(X_train[:,int(max(markers[0]-h,0)):int(min(markers[0]+h,X_train.shape[1]-1))])
X_test_ldv = lda.transform(X_test[:,int(max(markers[0]-h,0)):int(min(markers[0]+h,X_test.shape[1]-1))])
for i in range(1,m):
    lda.fit(X_train[:,int(max(markers[i]-h,0)):int(min(markers[i]+h,X_train.shape[1]-1))],Y_train)
    X_train_new = lda.transform(X_train[:,int(max(markers[i]-h,0)):int(min(markers[i]+h,X_train.shape[1]-1))])
    X_test_new = lda.transform(X_test[:,int(max(markers[i]-h,0)):int(min(markers[i]+h,X_test.shape[1]-1))])
    X_train_ldv=np.hstack((X_train_ldv,X_train_new))
    X_test_ldv=np.hstack((X_test_ldv,X_test_new))
#print(X_ldv)
#print(np.shape(X_ldv))

In [18]:
clf = SVC(kernel='rbf',decision_function_shape='ovo')
clf.fit(X_train_ldv, Y_train)
print(clf.score(X_train_ldv, Y_train))
print(clf.score(X_test_ldv, Y_test))


0.964
0.914790538615
